In [ ]:
import pandas as pd

# Load the dataset
file_path = '/content/drive/MyDrive/Assessment 2(Programming)/books_1.Best_Books_Ever.csv'
books_df = pd.read_csv(file_path)

# Display the first few rows of the dataset
books_df.head()


In [20]:
# Filtering the dataset for books in foreign languages (English)
foreign_language_books_df = books_df[books_df['language'] == 'English']

In [ ]:
import re
# Function to check if the description suggests the book is suitable for children
def is_description_suitable(description):
    if not isinstance(description, str):
        return False  # If the description is missing or not a string, we'll consider it unsuitable

    # List of keywords that might indicate unsuitability for young children
    unsuitable_keywords = ['Mature', 'Violence', 'Sexual', 'Explicit', 'Horror']

    # Basic check for the presence of these keywords
    if any(re.search(keyword, description, re.IGNORECASE) for keyword in unsuitable_keywords):
        return False
    return True

# Apply the updated function to further filter books
suitable_children_books_df = foreign_language_books_df[foreign_language_books_df['description'].apply(is_description_suitable)]

# Display filtered dataset
suitable_children_books_df


In [ ]:
# Filter books based on a high rating(>=4.0) and enough people commented(>=200000) and  a high enough positive rating(>=95.0)
highly_rated_children_books_df = suitable_children_books_df[(suitable_children_books_df['rating'] >= 4.0) & (suitable_children_books_df['numRatings'] >= 200000) &(suitable_children_books_df['likedPercent'] >= 95.0)]
# Display the dataset with highly selected books
highly_rated_children_books_df


In [ ]:
# Considering books with a known price and below a certain threshold (e.g. $20)
# Convert the 'price' column to numeric values
# This step will handle any non-numeric values by converting them to NaN
highly_rated_children_books_df['price'] = pd.to_numeric(highly_rated_children_books_df['price'], errors='coerce')

# Run this filter with the corrected price column
affordable_books = highly_rated_children_books_df[(highly_rated_children_books_df['price'] <= 20)]

# Display the final selection of books
affordable_books
# affordable_books(164 rows × 25 columns)

In [24]:
# At this step I need to see the complete list, so I need to print it out
affordable_books.to_csv('affordable_books_selection.csv', index=False)

In [25]:
import pandas as pd
# Here I made a file myself and manually entered the Lexile value of each book
# Considering that this is a dataset from the United States, I think it is reasonable to further screen the books according to the American Lexile criteria.
# But I didn't find any relevant data sets online, so I manually entered the book names filtered out in the previous step on the Lexile website, and finally made a table(Lexile_score.csv).
file_path_2 = '/content/drive/MyDrive/Assessment 2(Programming)/Lexile_score.csv'
Lexile_df = pd.read_csv(file_path_2)

# Merge the two tables using the common 'title' column
merge_df = pd.merge(affordable_books, Lexile_df, on='title', how='left')

# Because it is a manual search, some books do not have corresponding Lexile values (NaN).
cleaned_merged_books_df = merge_df.dropna(subset=['Lexile'])

cleaned_merged_books_df.to_csv('cleaned_merged_books.csv', index=False)

In [ ]:
# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns
# Visualization 1 Top10 Most rated books
top10_rated_books = cleaned_merged_books_df.nlargest(10, 'numRatings')
plt.figure(figsize =(12,8))
sns.barplot(x='numRatings', y='title', data= top10_rated_books, palette="viridis")
plt.title('Top10 Most Rated Books')
plt.xlabel('Number of Ratings')
plt.ylabel('Books title')
plt.grid(True)
plt.show()

In [ ]:
# Visualization 2 Top10 Genres distribution
import ast
from collections import Counter
genre_list = []
for genres in cleaned_merged_books_df['genres']:
  genres = ast.literal_eval(genres)
  genre_list.extend(genres)
genre_counts = Counter(genre_list)
top_genres = dict(genre_counts.most_common(10))
plt.figure(figsize=(10, 8))
plt.pie(top_genres.values(), labels=top_genres.keys(), autopct='%1.1f%%', startangle=140)
plt.title('Top 10 Genres Distribution')
plt.show()

In [ ]:
# Visualization 3 Average ratings by Top10 Authors
author_counts = cleaned_merged_books_df['author'].value_counts()
authors_with_multiple_books = author_counts[author_counts > 1].index
filtered_books = books_df[books_df['author'].isin(authors_with_multiple_books)]
average_ratings = filtered_books.groupby('author')['rating'].mean().sort_values(ascending=False).head(10)
plt.figure(figsize=(12, 8))
sns.barplot(x=average_ratings.values, y=average_ratings.index, palette="mako")
plt.title('Average Rating by Author (Top 10 Authors with Multiple Books)')
plt.xlabel('Average Rating')
plt.ylabel('Author')
plt.grid(True)
plt.show()

In [ ]:
# Visualization 4 Distribution of Book Ratings
plt.figure(figsize=(10, 6))
sns.histplot(cleaned_merged_books_df['rating'], bins=30, kde=False, color='skyblue')
plt.title('Distribution of Book Ratings')
plt.xlabel('Ratings')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

In [ ]:
# Visualization 5 Distribution of Books' price
cleaned_merged_books_df['price'] = pd.to_numeric(cleaned_merged_books_df['price'], errors='coerce')
books_with_price = cleaned_merged_books_df.dropna(subset=['price'])
plt.figure(figsize=(10, 6))
sns.histplot(books_with_price['price'], bins=30, color='purple', kde=True)
plt.title('Price Distribution of Books')
plt.xlabel('Price (USD)')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

In [ ]:
# Visualization 6: Book Ratings & Number of Ratings
plt.figure(figsize=(10, 6))
sns.scatterplot(data=cleaned_merged_books_df, x='rating', y='numRatings', alpha=0.5)
plt.title('Book Ratings & Number of Ratings')
plt.xlabel('Rating')
plt.ylabel('Number of Ratings')
plt.grid(True)
plt.show()